## PDF Ingestion

In [3]:
from langchain_community.document_loaders import PyPDFLoader

local_path = "WEF_The_Global_Cooperation_Barometer_2024.pdf"

loader = PyPDFLoader(local_path)
pages = loader.load()


## Vector DB Setup

In [5]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

#Split and Chunk
text_splitter = RecursiveCharacterTextSplitter(chunk_size=7500, chunk_overlap=100)
chunks = text_splitter.split_documents(pages)


In [6]:
!ollama list

NAME                   	ID          	SIZE  	MODIFIED       
nomic-embed-text:latest	0a109f422b47	274 MB	22 minutes ago	
llama3.1:latest        	42182419e950	4.7 GB	17 hours ago  	


In [9]:
#add to vetcor db
vector_db = Chroma.from_documents(
    documents=chunks,
    embedding=OllamaEmbeddings(model='nomic-embed-text', show_progress=True),
    collection_name='local_rag'
)

OllamaEmbeddings: 100%|██████████| 26/26 [00:03<00:00,  8.57it/s]


## Retrieval

In [10]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

In [11]:
MODEL = 'llama3.1'
model = ChatOllama(model=MODEL)

In [12]:
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate five
    different versions of the given user question to retrieve relevant documents from
    a vector database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}""",
)

In [13]:
retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(),
    model,
    prompt=QUERY_PROMPT
)

template = """
Answer the question below based on the the context, If you 
don't know the answer, replay that I don't know the answer. 

Context : {context}
Question : {question}
"""
prompt = ChatPromptTemplate.from_template(template=template)

In [14]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [15]:
chain.invoke(input(""))

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 45.37it/s]


'This article appears to be a summary of the Global Cooperation Barometer 2024 report by the World Economic Forum (WEF). It discusses trends and developments in global cooperation, particularly in the areas of trade and capital flows. The article highlights the moderate growth in cooperation between 2012-2020, followed by pandemic-related disruptions and subsequent recovery. It also explores questions around inclusive potential, trade barriers, and the need for leaders to rebuild economic connections to promote growth, diversity, resilience, and social inclusion.\n\nSome specific points discussed in the article include:\n\n* The increase in foreign direct investment (FDI) stock from 30% of GDP in 2012 to 49% in 2020\n* The mixed story of trade, with services flows rising but goods flows declining between 2012-2020\n* The rebounding of trade after the initial pandemic shock, with goods flows reaching a historical peak at 51% of GDP in 2022\n* The normalization of capital flows after an 

In [16]:
chain.invoke("What are the 5 pillars of global cooperation?")

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 71.74it/s]


'The text does not explicitly mention the "5 pillars" of global cooperation. However, it does discuss the trade and capital pillar in some detail.\n\nIn the provided snippet, there is a discussion about growth in global cooperation across different sectors, but the term "pillars" itself is not used to categorize these sectors or areas of cooperation.\n\nIf you\'re looking for information on the 5 pillars of global cooperation, I\'d be happy to try and help you find it. However, based on this text alone, we can only infer that one specific pillar (trade and capital) has been discussed.'